In [5]:
# Imports

import os
import requests
import json
import os
from dotenv import load_dotenv

In [12]:
# Setup

load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")

API_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
}

In [13]:
# Main 

def call_openrouter(message, model):

    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": message}
        ]
    }
    
    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))

    if response.status_code == 200:
        data = response.json()
        return data["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")



def agent(prompt, model):

    print("Agent starting...")
    print("Sending prompt:", prompt)
    
    try:
        result = call_openrouter(prompt, model)
        print("\nResponse from OpenRouter:\n")
        print(result)
    except Exception as e:
        print("\nError during API call:\n", e)


In [14]:
gemini_2_flash_lite = 'google/gemini-2.0-flash-lite-preview-02-05:free'

gemini_2_pro_exp = 'google/gemini-2.0-pro-exp-02-05:free'

prompt = "Tell me about yourself!"

In [ ]:
agent(prompt, gemini_2_flash_lite)